In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
       
    url= "https://www.glasdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword"+keyword+"&sc.keyword=data+Scientist&locT=&locId=&jobType="    #<-Modification made by Ken Jee

    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)
        time.sleep(.1)
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("react-job-listing")  #looks like this works. Buy since I don't understand what I'm doing....
        for job_button in job_buttons:  
                
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))

            if len(jobs) >= num_jobs:
                break
  
            job_button.click()  #You might 
     
            time.sleep(1)
    
            collected_successfully = False
 
            try:
                driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.    <- Again Ken Jee magic
            except NoSuchElementException:
                pass
            
            while not collected_successfully:

                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "css-1j389vi e1tk4kwz2")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                    
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-1hbqxax e1wijj240"]').text#    <- Again Ken Jee magic
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@class="css-1m5m32b e1tk4kwz4"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            try:
                driver.find_element_by_xpath('.//div[@data-item="tab" and @data-tab-type="overview"]').click()

                try:
                    size = driver.find_element_by_xpath('.//span[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//span[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//span[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//span[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//span[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//span[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//a[@data-test="pagination-next"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame. 

In [3]:
import glassdoor_scraper as gs
path="/usr/local/bin/chromedriver"
df = gs.get_jobs("data scientist", 100, False,path,5)
df


Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,$64K - $107K (Glassdoor est.),Job Description\nResponsible for overseeing th...,4.1,CarShield\n4.1,"O'Fallon, MO",1001 to 5000 Employees,2005,Company - Public,Consumer Product Rental,Consumer Services,Unknown / Non-Applicable
1,Data Scientist,Employer Provided Salary:$150K,We are adding a Data Scientist to our growing ...,4.3,American Capital Group\n4.3,"Bellevue, WA",51 to 200 Employees,1986,Company - Private,Real Estate,Real Estate,Unknown / Non-Applicable
2,Data Scientist,$64K - $120K (Glassdoor est.),"At Ezoic, we build a platform to help online p...",4.5,Ezoic\n4.5,"Carlsbad, CA",51 to 200 Employees,2010,Company - Private,Research & Development,Business Services,$100 to $500 million (USD)
3,BI Data Engineer/ Data Scientist,$70K - $123K (Glassdoor est.),You're considering a career with one of the la...,3.6,Golden State Foods\n3.6,"Irvine, CA",5001 to 10000 Employees,1947,Company - Private,Wholesale,Business Services,$5 to $10 billion (USD)
4,Data Scientist,$56K - $98K (Glassdoor est.),What We Offer:\nAt Blain’s Farm & Fleet we bel...,3.4,Blain's Farm & Fleet\n3.4,"Janesville, WI",1001 to 5000 Employees,1955,Company - Private,Other Retail Stores,Retail,Unknown / Non-Applicable
5,Senior Data Scientist,Employer Provided Salary:$90K - $110K,Who we are?\nTricolor Holdings is an integrate...,3.5,Tricolor Auto Group\n3.5,"Irving, TX",501 to 1000 Employees,2007,Company - Private,Vehicle Dealers,Retail,Unknown / Non-Applicable
6,Data Scientist,$71K - $116K (Glassdoor est.),Title: Data Scientist\nDepartment: Software De...,4.2,Akoya Biosciences\n4.2,"Marlborough, MA",51 to 200 Employees,-1,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable
7,Data Scientist,$69K - $126K (Glassdoor est.),"Systems and Manafucturing, A division of SHINE...",3.7,SHINE Medical Technologies\n3.7,"Fitchburg, WI",51 to 200 Employees,2010,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Less than $1 million (USD)
8,Data Scientist - Credit/Fraud Strategy,$76K - $161K (Glassdoor est.),"Fora Financial, located in NYC, is a leading f...",3.4,Fora Financial LLC\n3.4,"New York, NY",51 to 200 Employees,2008,Company - Private,Investment Banking & Asset Management,Finance,$50 to $100 million (USD)
9,Data Scientist,Employer Provided Salary:$145K - $175K,Geospatial Data Scientist\nGeospatial Data Sci...,-1,EpochGeo,"Reston, VA",-1,-1,-1,-1,-1,-1
